<a href="https://colab.research.google.com/github/aromanenko/ATSF/blob/wip/hw3_solution_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import libs

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
# %matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
pd.options.plotting.backend = "plotly"

# upload and investigate data

In [123]:
# upload train.csv file,
# it contains train and test samplesa
all_data = pd.read_csv('train.csv', delimiter=',')

# connvert date-column to data format
all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'], format= "%Y-%m-%d")
all_data.head()

,Unnamed: 0,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# rename "Unnamed: 0" to id (it's needed to manage train/and)
all_data.rename(columns={'Unnamed: 0': 'id'}, inplace=True)
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0
35340,35538,40372,1380,2019-12-30,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0
35341,35539,40373,1380,2019-12-30,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0
35342,35540,46272,1380,2019-12-30,NaN,1.0,0.0,284.29,199.0,0.0,1.0


In [125]:
# investigate data holiscitly
all_data.describe()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG
count,35344.000000,35344.000000,35344.000000,35344,34144.000000,35159.000000,35159.0,34217.000000,34212.000000,35159.0,35159.000000
mean,17766.554012,49253.732232,844.240154,2018-07-24 16:17:19.746491904,12.245636,0.206434,0.0,1167.679357,1155.778351,0.0,0.907677
min,0.000000,40369.000000,309.000000,2016-12-19 00:00:00,0.000000,0.000000,0.0,49.000000,8.647059,0.0,0.000000
25%,8881.750000,40370.000000,535.000000,2017-11-06 00:00:00,2.000000,0.000000,0.0,284.290000,199.000000,0.0,1.000000
50%,17770.500000,40372.000000,862.000000,2018-07-30 00:00:00,6.000000,0.000000,0.0,1000.000000,1000.000000,0.0,1.000000
75%,26647.250000,46272.000000,1173.000000,2019-04-22 00:00:00,12.000000,0.000000,0.0,2000.000000,2000.000000,0.0,1.000000
max,35541.000000,96212.000000,1380.000000,2019-12-30 00:00:00,1160.000000,2.000000,0.0,3000.000000,3000.000000,0.0,1.000000
std,10258.040738,19145.064867,333.229160,NaN,32.604642,0.433393,0.0,1046.828551,1057.912830,0.0,0.289486


In [126]:
# draw some time series
def plot_some_ts(ts_df, groupby_columns, time_column, target_column, ts_num = 10, aggregation_method = 'sum'):
  '''pivot original ts by group_columns
     ts_df - original dataframe with ts data,
     group_columns - tuple with names of columns used to split data to time series, use None if splitting is not needed
     time_column - name of column with date, datetime64
     target_column - column with ts data (data should be in numerical format), string
     ts_num - number of ts to be drawn, int
     aggregation_method - aggregation method of data in target column, string
  '''

  if groupby_columns is None:
    ts_df[target_column + time_column + 'const'] = 1
    groupby_columns = [target_column + time_column + 'const']

  pivot_ts = ts_df.groupby(groupby_columns + [time_column]).agg(aggregation_method)

  # concat multiple index to single column
  index_column_name = ', '.join([groupby_columns[i]+'={0['+str(i)+']}' for i in range(len(groupby_columns))])
  pivot_ts.index = [pivot_ts.index.map(index_column_name.format) , pivot_ts.index.get_level_values(len(groupby_columns))]

  # unstack by-column (column that contains ts name)
  pivot_ts = pivot_ts.unstack([0])[target_column]

  # plot first ts_num ts
  fig = pivot_ts[pivot_ts.columns[:ts_num]].plot().update_layout(height=350, width=1300,
                                                  title="first {0} ts for {1} variable".format(ts_num, target_column ),
    xaxis_title=time_column,
    yaxis_title=target_column+ ' value',
    legend_title='ts id columns: '+', '.join(groupby_columns)).show()
  return fig

# data for 3 products (all stores-product level)
plot_some_ts(all_data, ['product_rk'], 'period_start_dt', 'demand', ts_num = 3)

In [127]:
# all stores - all products level
# pay attention to forecasting periods: since 2Dec2019
plot_some_ts(all_data, None, 'period_start_dt', 'demand', ts_num = 1)

# what data dependencies can you observe?

In [128]:
# investigate demand driver columns (explanatory variables )
print(all_data['PROMO1_FLAG'].unique())
print(all_data['PROMO2_FLAG'].unique()) #не используем данную переменную, т.к. значения только 0 и NaN
print(all_data['NUM_CONSULTANT'].unique()) #не используем данную переменную, т.к. значения только 0 и NaN
print(all_data['AUTORIZATION_FLAG'].unique())

[nan  1.  0.  2.]
[nan  0.]
[nan  0.]
[nan  1.  0.]


In [129]:
# remove those, which have only one unique value (not empty) (they do not provide any benefit when training the model)
del all_data['PROMO2_FLAG']
del all_data['NUM_CONSULTANT']
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst
0,0,40369,309,2016-12-19,29.0,NaN,NaN,NaN,NaN,1
1,1,40370,309,2016-12-19,64.0,NaN,NaN,NaN,NaN,1
2,2,40372,309,2016-12-19,32.0,NaN,NaN,NaN,NaN,1
3,3,40373,309,2016-12-19,10.0,NaN,NaN,NaN,NaN,1
4,4,46272,309,2016-12-19,15.0,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0,1
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0,1
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0,1
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0,1


# Preprocess data

In [130]:
# massage missing values, fill them based on common-sense
all_data['PROMO1_FLAG'] = all_data['PROMO1_FLAG'].fillna(all_data['PROMO1_FLAG'].mode()[0]) # most frequent value
all_data

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,NaN,1
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,NaN,1
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,NaN,1
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,NaN,1
4,4,46272,309,2016-12-19,15.0,0.0,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,2019-12-30,NaN,0.0,1000.00,1000.0,1.0,1
35340,35538,40372,1380,2019-12-30,NaN,0.0,2000.00,2000.0,1.0,1
35341,35539,40373,1380,2019-12-30,NaN,0.0,3000.00,3000.0,1.0,1
35342,35540,46272,1380,2019-12-30,NaN,1.0,284.29,199.0,1.0,1


## Fill missing values with prev/back info

In [131]:
# all_data[all_data['store_location_rk'] == 309].groupby('period_start_dt').sum().plot()a

In [64]:
all_data.head()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst
0,0,40369,309,2016-12-19,29.0,0.0,NaN,NaN,1.0,1
1,1,40370,309,2016-12-19,64.0,0.0,NaN,NaN,1.0,1
2,2,40372,309,2016-12-19,32.0,0.0,NaN,NaN,1.0,1
3,3,40373,309,2016-12-19,10.0,0.0,NaN,NaN,1.0,1
4,4,46272,309,2016-12-19,15.0,0.0,NaN,NaN,1.0,1


In [132]:
# Fill missing values on store-product level PRICE_REGULAR, PRICE_AFTER_DISC, AUTHORIZATION_FLAG
# idea: price in week W is quite close to price in week W-1
all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt']).unstack([0,1])['PRICE_REGULAR']

# dell data relating to store_id = 309 because there is no price data at all
# all_data[all_data['store_location_rk'] == 309].sort_values(by='product_rk')

product_rk,40369,40370,40372,40373,46272,40369,40370,40372,40373,46272,...,40372,40373,46272,96212,40369,40370,40372,40373,96212,46272
store_location_rk,309,309,309,309,309,317,317,317,317,317,...,1363,1363,1363,1363,1380,1380,1380,1380,1380,1380
period_start_dt,,,,,,,,,,,,,,,,,,,,,
2016-12-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-12-26,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,157.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,157.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-09,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,157.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-16,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,157.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-02,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,284.29,...,2000.0,3000.0,284.29,141.43,500.0,1000.0,2000.0,3000.0,141.43,284.29
2019-12-09,NaN,NaN,NaN,NaN,NaN,500.0,1000.0,2000.0,3000.0,284.29,...,2000.0,3000.0,284.29,141.43,500.0,1000.0,2000.0,3000.0,141.43,284.29


In [117]:
# all_data[(all_data['store_location_rk'] == 317) & (all_data['product_rk'] == 40369)]['PRICE_REGULAR']

In [40]:
# AUTORIZATION_FLAG - is product available at store at the moment
plot_some_ts(all_data, ['product_rk', 'store_location_rk'], 'period_start_dt', 'AUTORIZATION_FLAG', ts_num = 3)

In [133]:
# fill na with prev (and if no prev then next) value
all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
  .unstack([0,1])\
   ['PRICE_REGULAR'].\
   ffill().bfill().\
   stack([1,0]).\
   rename('REGULAR_PRICE_FIXED')

/tmp/ipython-input-2466284687.py:6: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



period_start_dt  store_location_rk  product_rk
2016-12-19       317                40369          500.00
                                    40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    46272          157.00
                                                   ...   
2019-12-30       1380               40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    46272          284.29
                                    96212          141.43
Name: REGULAR_PRICE_FIXED, Length: 38160, dtype: float64

In [134]:
# fill na with prev (and if no prev then next) values
all_data = all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt']).\
  merge( all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
            .unstack([0,1])\
            ['PRICE_REGULAR'].\
              ffill().bfill().\
            stack([1,0]).\
            rename('PRICE_REGULAR_FIXED'),
         how = 'left', right_index = True, left_index = True)\
  .reset_index()

# dell original column and replace with new one
del all_data['PRICE_REGULAR']
all_data.rename(columns = {'PRICE_REGULAR_FIXED':'PRICE_REGULAR'}, inplace=True)
all_data.head()

/tmp/ipython-input-2283269789.py:7: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst,PRICE_REGULAR
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,1,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,1,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,1,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,1,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,1,NaN


In [140]:
def ts_fillna_ffill_bfill(ts_df,column_name, ts_id):
#  all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\ # define id columns
#   .unstack([0,1])\                          # df -> pivot transformation: date column - is row-index, product x store - is column index
#   ['PRICE_REGULAR'].\                      # define column with data to be fixed
#   ffill().bfill().\                        # fill missing value: apply forwand then back filling method consequently
#   stack([1,0]).\                           # pivot -> ts transformation
#   rename('REGULAR_PRICE_FIXED')            # rename column


  # fill na with prev (and if no prev then next) values
  new_ts_df = ts_df.set_index(ts_id).\
    merge(ts_df.set_index(ts_id)\
              .unstack([0,1])\
              [column_name].\
              ffill().bfill().\
              stack([1,0]).\
              rename(column_name),
          how = 'left', right_index = True, left_index = True)\
    .reset_index()

  # dell original column and replace with new one
  del new_ts_df[column_name+'_x']
  return new_ts_df.rename(columns = {column_name+'_y':column_name})

/tmp/ipython-input-1406430599.py:16: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst,PRICE_REGULAR
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,1,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,1,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,1,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,1,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,1,NaN
...,...,...,...,...,...,...,...,...,...,...
35339,40370,1380,2019-12-30,35537,NaN,0.0,1000.0,1.0,1,1000.00
35340,40372,1380,2019-12-30,35538,NaN,0.0,2000.0,1.0,1,2000.00
35341,40373,1380,2019-12-30,35539,NaN,0.0,3000.0,1.0,1,3000.00
35342,46272,1380,2019-12-30,35540,NaN,1.0,199.0,1.0,1,284.29


In [142]:
all_data.head()

,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,PRICE_AFTER_DISC,AUTORIZATION_FLAG,demandperiod_start_dtconst,PRICE_REGULAR
0,40369,309,2016-12-19,0,29.0,0.0,NaN,NaN,1,NaN
1,40370,309,2016-12-19,1,64.0,0.0,NaN,NaN,1,NaN
2,40372,309,2016-12-19,2,32.0,0.0,NaN,NaN,1,NaN
3,40373,309,2016-12-19,3,10.0,0.0,NaN,NaN,1,NaN
4,46272,309,2016-12-19,4,15.0,0.0,NaN,NaN,1,NaN


In [143]:
# the same fort PRICE_AFTER_DISCOUNT
all_data = ts_fillna_ffill_bfill(ts_df = all_data,column_name = 'PRICE_AFTER_DISC' , ts_id= ['product_rk', 'store_location_rk', 'period_start_dt'])


# the same fort AUTORIZATION_FLAG
all_data = ts_fillna_ffill_bfill(ts_df = all_data,column_name = 'AUTORIZATION_FLAG' , ts_id= ['product_rk', 'store_location_rk', 'period_start_dt'])

/tmp/ipython-input-1406430599.py:16: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

/tmp/ipython-input-1406430599.py:16: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [144]:
# look at data again
all_data.head()

,product_rk,store_location_rk,period_start_dt,id,demand,PROMO1_FLAG,demandperiod_start_dtconst,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
0,40369,309,2016-12-19,0,29.0,0.0,1,NaN,NaN,NaN
1,40370,309,2016-12-19,1,64.0,0.0,1,NaN,NaN,NaN
2,40372,309,2016-12-19,2,32.0,0.0,1,NaN,NaN,NaN
3,40373,309,2016-12-19,3,10.0,0.0,1,NaN,NaN,NaN
4,46272,309,2016-12-19,4,15.0,0.0,1,NaN,NaN,NaN


## Fill missing values as average/mode/median from other stores

In [115]:
# find mean values for each pair product x date
values = all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
            .unstack([0,2])\
            ['PRICE_REGULAR'].\
              mean()

# replace missing values with mean in all stores
all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\
            .unstack([0,2])\
            ['PRICE_REGULAR'].\
            fillna(value = values).\
            stack([1,0]).\
            rename('REGULAR_PRICE_FIXED')

/tmp/ipython-input-3031208545.py:10: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



store_location_rk  period_start_dt  product_rk
309                2016-12-26       40369          500.00
                                    40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    46272          157.00
                                                   ...   
1380               2019-12-30       40370         1000.00
                                    40372         2000.00
                                    40373         3000.00
                                    46272          284.29
                                    96212          141.43
Name: REGULAR_PRICE_FIXED, Length: 37679, dtype: float64

In [145]:
def ts_fillna_aggmethod(ts_df,column_name, ts_id):
#  all_data.set_index(['product_rk', 'store_location_rk', 'period_start_dt'])\ # define id columns
#   .unstack([0,1])\                          # df -> pivot transformation: date column - is row-index, product x store - is column index
#   ['PRICE_REGULAR'].\                      # define column with data to be fixed
#   ffill().bfill().\                        # fill missing value: apply forwand then back filling method consequently
#   stack([1,0]).\                           # pivot -> ts transformation
#   rename('REGULAR_PRICE_FIXED')            # rename column

  values = all_data.set_index(ts_id)\
            .unstack([0,1])\
            [column_name].\
              mean()

  # fill na with prev (and if no prev then next) values
  new_ts_df = ts_df.set_index(ts_id).\
    merge(ts_df.set_index(ts_id)\
              .unstack([0,1])\
              [column_name].\
              fillna(value = values).\
              stack([1,0]).\
              rename(column_name),
          how = 'left', right_index = True, left_index = True)\
    .reset_index()

  # dell original column and replace with new one
  del new_ts_df[column_name+'_x']
  return new_ts_df.rename(columns = {column_name+'_y':column_name})


# for PRICE_REGULAR
all_data = ts_fillna_aggmethod(ts_df = all_data,column_name = 'PRICE_AFTER_DISC' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])


/tmp/ipython-input-1361380021.py:20: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [146]:
# the same fort PRICE_AFTER_DISCOUNT
all_data = ts_fillna_aggmethod(ts_df = all_data,column_name = 'PRICE_AFTER_DISC' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])


# the same fort AUTORIZATION_FLAG
all_data = ts_fillna_aggmethod(ts_df = all_data,column_name = 'AUTORIZATION_FLAG' , ts_id= ['product_rk', 'period_start_dt', 'store_location_rk'])

/tmp/ipython-input-1361380021.py:20: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

/tmp/ipython-input-1361380021.py:20: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [147]:
# check data again
all_data.isna().sum()

,0
product_rk,0
period_start_dt,0
store_location_rk,0
id,0
demand,1200
PROMO1_FLAG,0
demandperiod_start_dtconst,0
PRICE_REGULAR,15
PRICE_AFTER_DISC,0
AUTORIZATION_FLAG,0


In [148]:
# let's delete store 309 related data
all_data = all_data[all_data['store_location_rk'] != 309]
all_data

,product_rk,period_start_dt,store_location_rk,id,demand,PROMO1_FLAG,demandperiod_start_dtconst,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
15,40369,2016-12-19,317,15,50.0,0.0,1,500.00,500.0,1.0
16,40370,2016-12-19,317,16,44.0,0.0,1,1000.00,1000.0,1.0
17,40372,2016-12-19,317,17,13.0,0.0,1,2000.00,2000.0,1.0
18,40373,2016-12-19,317,18,6.0,0.0,1,3000.00,3000.0,1.0
19,46272,2016-12-19,317,19,34.0,0.0,1,157.00,157.0,1.0
...,...,...,...,...,...,...,...,...,...,...
35339,40370,2019-12-30,1380,35537,NaN,0.0,1,1000.00,1000.0,1.0
35340,40372,2019-12-30,1380,35538,NaN,0.0,1,2000.00,2000.0,1.0
35341,40373,2019-12-30,1380,35539,NaN,0.0,1,3000.00,3000.0,1.0
35342,46272,2019-12-30,1380,35540,NaN,1.0,1,284.29,199.0,1.0


In [150]:
# check data again
# that's it
all_data.isna().sum()

,0
product_rk,0
period_start_dt,0
store_location_rk,0
id,0
demand,1200
PROMO1_FLAG,0
demandperiod_start_dtconst,0
PRICE_REGULAR,0
PRICE_AFTER_DISC,0
AUTORIZATION_FLAG,0


In [ ]:
#Удалим также такие пары "товар+день", у которых вообще не известна цена ни в одном магазине. Выведем для начала такие пары (их пять)
n = 0
listt = []
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    if len(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]) == all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].isnull().sum():
      n += 1
      listt.append([i, d])

print(n)
print(listt)

5
[[40369, Timestamp('2016-12-19 00:00:00')], [40370, Timestamp('2016-12-19 00:00:00')], [40372, Timestamp('2016-12-19 00:00:00')], [40373, Timestamp('2016-12-19 00:00:00')], [46272, Timestamp('2016-12-19 00:00:00')]]


In [ ]:
all_data = all_data[(all_data['product_rk'] != 40369) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40370) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40372) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 40373) | (all_data['period_start_dt'] != '2016-12-19')]
all_data = all_data[(all_data['product_rk'] != 46272) | (all_data['period_start_dt'] != '2016-12-19')]

In [ ]:
#Проверка
n = 0
listt = []
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    if len(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]) == all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].isnull().sum():
      n += 1
      listt.append([i, d])

print(n)
print(listt)
#Таким образом, на 2016-12-19 остался только один товар №96212

0
[]


In [ ]:
#Проверка:
a = 0
for i in all_data['product_rk'].unique():
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    a += 1

a #919 пар вместо 924

919

In [ ]:
#Выведем для проверки, что были NaN в переменных PRICE_REGULAR и PRICE_AFTER_DISC, а после следующих двух ячеек проверим, что они исчезли
all_data[(all_data['product_rk'] == 96212) & (all_data['period_start_dt'] == '2019-11-04')]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
889,889,96212,317,2019-11-04,3.966667,1.0,70.0,49.000000,1.0
1812,1818,96212,355,2019-11-04,7.000000,1.0,70.0,49.000000,1.0
2735,2747,96212,380,2019-11-04,3.000000,1.0,70.0,49.000000,1.0
3659,3677,96212,425,2019-11-04,5.366667,1.0,70.0,49.000000,1.0
4582,4606,96212,453,2019-11-04,3.733333,1.0,70.0,49.000000,1.0
5501,5531,96212,504,2019-11-04,9.000000,1.0,70.0,49.000000,1.0
6425,6460,96212,517,2019-11-04,0.000000,1.0,70.0,49.000000,1.0
7348,7388,96212,525,2019-11-04,4.200000,1.0,70.0,49.000000,1.0
8272,8318,96212,533,2019-11-04,0.000000,0.0,NaN,NaN,0.0
9194,9240,96212,535,2019-11-04,4.000000,1.0,70.0,49.000000,1.0


In [ ]:
# fill nans in the price of product i (PRICE_REGULAR) on date d with the median price of this product for all stores in which it is sold on this date d
#Заполняем пропуски в цене товара i (PRICE_REGULAR) на дату d медианной ценой этого товара по всем магазинам, в которых он продается, на данную дату d
for i in all_data['product_rk'].unique():
  # print(i)
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    # print(d)
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].median())
    all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d), ['PRICE_REGULAR']] = all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].fillna(all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'].median())
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_REGULAR'])

In [ ]:
#Заполняем аналогично пропуски в переменной PRICE_AFTER_DISC
for i in all_data['product_rk'].unique():
  # print(i)
  for d in all_data[all_data['product_rk'] == i]['period_start_dt'].unique():
    # print(d)
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].median())
    all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d), ['PRICE_AFTER_DISC']] = all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].fillna(all_data.loc[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'].median())
    # print(all_data[(all_data['product_rk'] == i) & (all_data['period_start_dt'] == d)]['PRICE_AFTER_DISC'])

In [ ]:
#Выведем для проверки, что все NaN в переменных PRICE_REGULAR и PRICE_AFTER_DISC заменились на соответствующие медианы
all_data[(all_data['product_rk'] == 96212) & (all_data['period_start_dt'] == '2019-11-04')]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG
889,889,96212,317,2019-11-04,3.966667,1.0,70.0,49.000000,1.0
1812,1818,96212,355,2019-11-04,7.000000,1.0,70.0,49.000000,1.0
2735,2747,96212,380,2019-11-04,3.000000,1.0,70.0,49.000000,1.0
3659,3677,96212,425,2019-11-04,5.366667,1.0,70.0,49.000000,1.0
4582,4606,96212,453,2019-11-04,3.733333,1.0,70.0,49.000000,1.0
5501,5531,96212,504,2019-11-04,9.000000,1.0,70.0,49.000000,1.0
6425,6460,96212,517,2019-11-04,0.000000,1.0,70.0,49.000000,1.0
7348,7388,96212,525,2019-11-04,4.200000,1.0,70.0,49.000000,1.0
8272,8318,96212,533,2019-11-04,0.000000,0.0,70.0,49.000000,0.0
9194,9240,96212,535,2019-11-04,4.000000,1.0,70.0,49.000000,1.0


In [ ]:
#Проверка:
all_data['PRICE_REGULAR'].isnull().any()

False

In [ ]:
all_data.iloc[0,:].period_start_dt.day

26

In [ ]:
#перекодируем даты. Создадим три переменных ind_of_year, ind_of_month и ind_of_day, которые будут отражать год, месяц и день наблюдения соответственно

# all_data['period_start_dt'] = pd.to_datetime(all_data['period_start_dt'], dayfirst =False)
all_data["ind_of_year"] = [dt.year for dt in all_data.period_start_dt]
all_data["ind_of_month"] = [dt.month for dt in all_data.period_start_dt]
all_data["ind_of_day"] = [dt.day for dt in all_data.period_start_dt]
all_data.head()

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,20,40369,317,2016-12-26,65.0,1.0,500.0,500.0,1.0,2016,12,26
21,21,40370,317,2016-12-26,83.0,1.0,1000.0,1000.0,1.0,2016,12,26
22,22,40372,317,2016-12-26,30.0,1.0,2000.0,2000.0,1.0,2016,12,26
23,23,40373,317,2016-12-26,7.0,1.0,3000.0,3000.0,1.0,2016,12,26
24,24,46272,317,2016-12-26,35.0,1.0,157.0,157.0,1.0,2016,12,26


In [ ]:
#пример
all_data.iloc[300:600]

,id,product_rk,store_location_rk,period_start_dt,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
320,320,40369,317,2018-01-15,12.000000,0.0,500.0,500.0,1.0,2018,1,15
321,321,40370,317,2018-01-15,8.000000,0.0,1000.0,1000.0,1.0,2018,1,15
322,322,40372,317,2018-01-15,4.000000,0.0,2000.0,2000.0,1.0,2018,1,15
323,323,40373,317,2018-01-15,2.000000,0.0,3000.0,3000.0,1.0,2018,1,15
324,324,46272,317,2018-01-15,3.000000,0.0,239.0,239.0,1.0,2018,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...
615,615,40370,317,2018-12-24,69.000000,0.0,1000.0,1000.0,1.0,2018,12,24
616,616,40372,317,2018-12-24,12.000000,0.0,2000.0,2000.0,1.0,2018,12,24
617,617,40373,317,2018-12-24,16.000000,0.0,3000.0,3000.0,1.0,2018,12,24
618,618,46272,317,2018-12-24,18.266667,1.0,329.0,98.7,1.0,2018,12,24


In [ ]:
#и теперь удалим переменную period_start_dt за ненадобностью
del all_data['period_start_dt']
all_data

,id,product_rk,store_location_rk,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,20,40369,317,65.0,1.0,500.00,500.0,1.0,2016,12,26
21,21,40370,317,83.0,1.0,1000.00,1000.0,1.0,2016,12,26
22,22,40372,317,30.0,1.0,2000.00,2000.0,1.0,2016,12,26
23,23,40373,317,7.0,1.0,3000.00,3000.0,1.0,2016,12,26
24,24,46272,317,35.0,1.0,157.00,157.0,1.0,2016,12,26
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,NaN,0.0,1000.00,1000.0,1.0,2019,12,30
35340,35538,40372,1380,NaN,0.0,2000.00,2000.0,1.0,2019,12,30
35341,35539,40373,1380,NaN,0.0,3000.00,3000.0,1.0,2019,12,30
35342,35540,46272,1380,NaN,1.0,284.29,199.0,1.0,2019,12,30


In [ ]:
#Подготовим данные для обучения модели
#Поделим данные на трейн и тест
data_train = all_data[all_data['demand'].isnull() == False]
data_train #получаем 33959 наблюдений

,id,product_rk,store_location_rk,demand,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,20,40369,317,65.0,1.0,500.000000,500.000000,1.0,2016,12,26
21,21,40370,317,83.0,1.0,1000.000000,1000.000000,1.0,2016,12,26
22,22,40372,317,30.0,1.0,2000.000000,2000.000000,1.0,2016,12,26
23,23,40373,317,7.0,1.0,3000.000000,3000.000000,1.0,2016,12,26
24,24,46272,317,35.0,1.0,157.000000,157.000000,1.0,2016,12,26
...,...,...,...,...,...,...,...,...,...,...,...
35309,35507,40370,1380,24.0,0.0,1000.000000,1000.000000,1.0,2019,11,25
35310,35508,40372,1380,11.0,0.0,2000.000000,2000.000000,1.0,2019,11,25
35311,35509,40373,1380,3.0,0.0,3000.000000,3000.000000,1.0,2019,11,25
35312,35510,46272,1380,0.0,1.0,284.290000,199.000000,1.0,2019,11,25


In [ ]:
data_test = all_data[all_data['demand'].isnull()]
#Сразу переименуем столбец "demand" в тестовой выборке на "predicted"
data_test.rename(columns={'demand': 'predicted'}, inplace=True)
data_test #получаем 1200 наблюдений

<ipython-input-55-bb62f74bfb70>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.rename(columns={'demand': 'predicted'}, inplace=True)


,id,product_rk,store_location_rk,predicted,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
908,908,40369,317,NaN,0.0,500.00,500.0,1.0,2019,12,2
909,909,40370,317,NaN,0.0,1000.00,1000.0,1.0,2019,12,2
910,910,40372,317,NaN,0.0,2000.00,2000.0,1.0,2019,12,2
911,911,40373,317,NaN,0.0,3000.00,3000.0,1.0,2019,12,2
912,912,46272,317,NaN,1.0,284.29,199.0,1.0,2019,12,2
...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,NaN,0.0,1000.00,1000.0,1.0,2019,12,30
35340,35538,40372,1380,NaN,0.0,2000.00,2000.0,1.0,2019,12,30
35341,35539,40373,1380,NaN,0.0,3000.00,3000.0,1.0,2019,12,30
35342,35540,46272,1380,NaN,1.0,284.29,199.0,1.0,2019,12,30


In [ ]:
X = data_train.drop(['id', 'demand'], axis=1)
y = data_train['demand']

In [ ]:
X

,product_rk,store_location_rk,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
20,40369,317,1.0,500.000000,500.000000,1.0,2016,12,26
21,40370,317,1.0,1000.000000,1000.000000,1.0,2016,12,26
22,40372,317,1.0,2000.000000,2000.000000,1.0,2016,12,26
23,40373,317,1.0,3000.000000,3000.000000,1.0,2016,12,26
24,46272,317,1.0,157.000000,157.000000,1.0,2016,12,26
...,...,...,...,...,...,...,...,...,...
35309,40370,1380,0.0,1000.000000,1000.000000,1.0,2019,11,25
35310,40372,1380,0.0,2000.000000,2000.000000,1.0,2019,11,25
35311,40373,1380,0.0,3000.000000,3000.000000,1.0,2019,11,25
35312,46272,1380,1.0,284.290000,199.000000,1.0,2019,11,25


In [ ]:
y

,demand
20,65.0
21,83.0
22,30.0
23,7.0
24,35.0
...,...
35309,24.0
35310,11.0
35311,3.0
35312,0.0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
#оптимальные параметры модели были подобраны вручную
#обучим модель и проверим ее качество
regressor = GradientBoostingRegressor(
    max_depth=11,
    n_estimators=420,
    learning_rate=0.2,
    random_state=1,
    min_samples_leaf=11,
    min_samples_split=2,
    loss='absolute_error'
)
regressor.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.2, loss='absolute_error',
                          max_depth=11, min_samples_leaf=11, n_estimators=420,
                          random_state=1)

In [ ]:
y_pred = regressor.predict(X_test)
mean_absolute_error(y_test, y_pred)

5.062950326564885

In [ ]:
#предскажем искомые значения спроса для нашей тестовой выборки (с декабря 2019)
X_test = data_test.drop(['id', 'predicted'], axis=1)
X_test

,product_rk,store_location_rk,PROMO1_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,AUTORIZATION_FLAG,ind_of_year,ind_of_month,ind_of_day
908,40369,317,0.0,500.00,500.0,1.0,2019,12,2
909,40370,317,0.0,1000.00,1000.0,1.0,2019,12,2
910,40372,317,0.0,2000.00,2000.0,1.0,2019,12,2
911,40373,317,0.0,3000.00,3000.0,1.0,2019,12,2
912,46272,317,1.0,284.29,199.0,1.0,2019,12,2
...,...,...,...,...,...,...,...,...,...
35339,40370,1380,0.0,1000.00,1000.0,1.0,2019,12,30
35340,40372,1380,0.0,2000.00,2000.0,1.0,2019,12,30
35341,40373,1380,0.0,3000.00,3000.0,1.0,2019,12,30
35342,46272,1380,1.0,284.29,199.0,1.0,2019,12,30


In [ ]:
#предскажем значения спроса для тестовой выборки
y_pred_res = regressor.predict(X_test)
y_pred_res

array([ 5.82334136,  8.14585683,  5.69535062, ..., 24.95657769,
        5.98159285,  7.92182532])

In [ ]:
#создадим датафрейм y_results, в который поместим результаты
y_results = data_test[['id', 'predicted']]
y_results['predicted'] = y_pred_res
y_results

<ipython-input-66-a28de4580432>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_results['predicted'] = y_pred_res


,id,predicted
908,908,5.823341
909,909,8.145857
910,910,5.695351
911,911,4.157089
912,912,4.604878
...,...,...
35339,35537,81.654309
35340,35538,39.476182
35341,35539,24.956578
35342,35540,5.981593


In [ ]:
# удалим отрицательыне прогнозы из выборки
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted
5520,-1.477845
5526,-0.143918
6473,-0.693504
24914,-0.096025
24915,-0.345976
24917,-0.425856
24923,-0.811523
31373,-0.563163
31379,-1.254942
31385,-1.987016


In [ ]:
#заменим все отрицательные предсказанные значения спроса на ноль
y_results.loc[y_results['predicted'] < 0, ['predicted']] = 0
y_results.loc[y_results['predicted'] < 0, ['predicted']]

,predicted


In [ ]:
#Записываем полученный датафрейм в csv файл:
y_results.to_csv('submission_example.csv',sep=',', encoding='utf-8', index=False)